In [ ]:
!pip install tensorflow_addons

     |████████████████████████████████| 1.1 MB 8.6 MB/s 


In [ ]:
from tensorflow.keras.optimizers import Adam
from keras.layers import Activation, MaxPool2D, Concatenate,Add
import keras as Ks
from keras.layers import Input
from glob import glob
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import tensorflow.keras.backend as K
import tensorflow as tf
from tensorflow.keras.metrics import Precision, Recall, MeanIoU
import cv2
from skimage import io
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint, LearningRateScheduler
from keras.models import Model
from keras.layers import Input
from keras.layers.core import Dense, Dropout, Activation, Flatten, Reshape, Permute
from keras.layers.convolutional import Convolution2D, MaxPooling2D, UpSampling2D, ZeroPadding2D
# import tensorflow as tf
from tensorflow.keras.layers import BatchNormalization
from keras.layers.merge import Multiply, Concatenate
from keras.utils import np_utils
# from keras.layers.pooling import MaxPoolingWithArgmax2D,MaxUnpooling2D
# import tensorflow as tf
from keras import backend as K
from keras.layers import Layer
# from Layer import  MaxUnpooling2D,MaxPoolingWithArgmax2D
# import tensorflow_addons as tfa
from tensorflow_addons.layers import MaxUnpooling2D
from keras import backend as K
from tensorflow.keras.layers import Input, Activation, BatchNormalization, Dropout, Lambda, Conv2D, Conv2DTranspose, MaxPooling2D, concatenate, AveragePooling2D, Dense, Flatten
from keras.layers import Layer

In [ ]:
class MaxPoolingWithArgmax2D(Layer):
    def __init__(self, pool_size=(2, 2), strides=(1, 1), padding='same', **kwargs):
        super(MaxPoolingWithArgmax2D, self).__init__(**kwargs)
        self.padding = padding
        self.pool_size = pool_size
        self.strides = strides

    def call(self, inputs, **kwargs):
        padding = self.padding
        pool_size = self.pool_size
        strides = self.strides
        if K.backend() == 'tensorflow':
            ksize = [1, pool_size[0], pool_size[1], 1]
            padding = padding.upper()
            strides = [1, strides[0], strides[1], 1]
            output, argmax = K.tf.nn.max_pool_with_argmax(inputs, ksize=ksize, strides=strides, padding=padding)
        else:
            errmsg = '{} backend is not supported for layer {}'.format(K.backend(), type(self).__name__)
            raise NotImplementedError(errmsg)
        argmax = K.cast(argmax, K.floatx())
        return [output, argmax]

    def compute_output_shape(self, input_shape):
        ratio = (1, 2, 2, 1)
        output_shape = [dim // ratio[idx] if dim is not None else None for idx, dim in enumerate(input_shape)]
        output_shape = tuple(output_shape)
        return [output_shape, output_shape]

    def compute_mask(self, inputs, mask=None):
        return 2 * [None]


# class MaxUnpooling2D(Layer):
#     def __init__(self, size=(1, 1), **kwargs):
#         super(MaxUnpooling2D, self).__init__(**kwargs)
#         self.size = size

#     def call(self, inputs, output_shape=None):
#         updates, mask = inputs[0], inputs[1]
#         with K.tf.variable_scope(self.name):
#             mask = K.cast(mask, 'int32')
#             input_shape = K.tf.shape(updates, out_type='int32')
#             #  calculation new shape
#             if output_shape is None:
#                 output_shape = (input_shape[0], input_shape[1] * self.size[0], input_shape[2] * self.size[1], input_shape[3])
#             self.output_shape1 = output_shape

#             # calculation indices for batch, height, width and feature maps
#             one_like_mask = K.ones_like(mask, dtype='int32')
#             batch_shape = K.concatenate([[input_shape[0]], [1], [1], [1]], axis=0)
#             batch_range = K.reshape(K.tf.range(output_shape[0], dtype='int32'), shape=batch_shape)
#             b = one_like_mask * batch_range
#             y = mask // (output_shape[2] * output_shape[3])
#             x = (mask // output_shape[3]) % output_shape[2]
#             feature_range = K.tf.range(output_shape[3], dtype='int32')
#             f = one_like_mask * feature_range

#             # transpose indices & reshape update values to one dimension
#             updates_size = K.tf.size(updates)
#             indices = K.transpose(K.reshape(K.stack([b, y, x, f]), [4, updates_size]))
#             values = K.reshape(updates, [updates_size])
#             ret = K.tf.scatter_nd(indices, values, output_shape)
#             return ret

#     def compute_output_shape(self, input_shape):
#         mask_shape = input_shape[1]
#         return mask_shape[0], mask_shape[1] * self.size[0], mask_shape[2] * self.size[1], mask_shape[3]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -U --no-cache-dir gdown --pre
!gdown --id 1-2HGO0znF0JAsgZ_et5Cr_owKVxr_bKZ

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for gdown: filename=gdown-4.4.0-py3-none-any.whl size=14774 sha256=cb6872d59172c20b0080b850aeee8ae5f90914c6aa0636ea074b515dd9e39096
  Stored in directory: /tmp/pip-ephem-wheel-cache-c9ar_hhr/wheels/fb/c3/0e/c4d8ff8bfcb0461afff199471449f642179b74968c15b7a69c
Successfully built gdown
  Attempting uninstall: gdown
    Found existing installation: gdown 4.2.2
    Uninstalling gdown-4.2.2:
      Successfully uninstalled gdown-4.2.2
/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1-2HGO0znF0JAsgZ_et5Cr_owKVxr_bKZ
To: /content/CT_Data.zip
100% 624M/624M [00:04<00:00, 143MB/s]


In [ ]:
from zipfile import ZipFile

file_name="/content/CT_Data.zip"

with ZipFile(file_name, 'r') as ziip:
  ziip.extractall()
  print('woow')

woow


In [ ]:
mask_files = glob('/content/content/drive/MyDrive/CT_Data/Masks/*')
len(mask_files)

7367

In [ ]:
CT = glob('/content/content/drive/MyDrive/CT_Data/CT_images/*')
len(CT)

7367

In [ ]:
df = pd.DataFrame({"image_path": CT, "mask_path":mask_files})


In [ ]:
Xtrain, Xval = train_test_split(df, test_size=0.15,random_state=101)
Xtest, Xval = train_test_split(Xval, test_size=0.5)
print("Train size is {}, valid size is {} & test size is {}".format(len(Xtrain), len(Xval), len(Xtest)))

Train size is 6261, valid size is 553 & test size is 553


In [ ]:
train_ids = list(Xtrain.image_path)
train_mask = list(Xtrain.mask_path)

val_ids = list(Xval.image_path)
val_mask= list(Xval.mask_path)

In [ ]:
def tversky(ytrue, ypred):
    ypredpos = K.flatten(ypred)
    ytruepos = K.flatten(ytrue)
    truepos = K.sum(ytruepos * ypredpos)
    falseneg = K.sum(ytruepos * (1-ypredpos))
    falsepos = K.sum((1-ytruepos)*ypredpos)
    alpha = 0.7
    smooth=100
    return (truepos + smooth)/(truepos + alpha*falseneg + (1-alpha)*falsepos + smooth)

def focaltversky(ytrue,ypred):
    ypred = tf.cast(ypred, tf.float32)
    ytrue = tf.cast(ytrue, tf.float32)
    
    pt_1 = tversky(ytrue, ypred)
    gamma = 0.75
    return K.pow((1-pt_1), gamma)

def tversky_loss(ytrue, ypred):
    return 1 - tversky(ytrue,ypred)

In [ ]:
from tensorflow.keras.models import Model

In [ ]:
input_shape=(256,256,3)
n_labels=3
kernel=3
pool_size=(2, 2)
output_mode="softmax"
inputs = Input(shape=input_shape)

    # encoder
conv_1 = Convolution2D(64, (kernel, kernel), padding="same")(inputs)
conv_1 = BatchNormalization()(conv_1)
conv_1 = Activation("relu")(conv_1)
conv_2 = Convolution2D(64, (kernel, kernel), padding="same")(conv_1)
conv_2 = BatchNormalization()(conv_2)
conv_2 = Activation("relu")(conv_2)

pool_1, mask_1 = MaxPoolingWithArgmax2D(pool_size)(conv_2)

conv_3 = Convolution2D(128, (kernel, kernel), padding="same")(pool_1)
conv_3 = BatchNormalization()(conv_3)
conv_3 = Activation("relu")(conv_3)
conv_4 = Convolution2D(128, (kernel, kernel), padding="same")(conv_3)
conv_4 = BatchNormalization()(conv_4)
conv_4 = Activation("relu")(conv_4)

pool_2, mask_2 = MaxPoolingWithArgmax2D(pool_size)(conv_4)

conv_5 = Convolution2D(256, (kernel, kernel), padding="same")(pool_2)
conv_5 = BatchNormalization()(conv_5)
conv_5 = Activation("relu")(conv_5)
conv_6 = Convolution2D(256, (kernel, kernel), padding="same")(conv_5)
conv_6 = BatchNormalization()(conv_6)
conv_6 = Activation("relu")(conv_6)
conv_7 = Convolution2D(256, (kernel, kernel), padding="same")(conv_6)
conv_7 = BatchNormalization()(conv_7)
conv_7 = Activation("relu")(conv_7)

pool_3, mask_3 = MaxPoolingWithArgmax2D(pool_size)(conv_7)

conv_8 = Convolution2D(512, (kernel, kernel), padding="same")(pool_3)
conv_8 = BatchNormalization()(conv_8)
conv_8 = Activation("relu")(conv_8)
conv_9 = Convolution2D(512, (kernel, kernel), padding="same")(conv_8)
conv_9 = BatchNormalization()(conv_9)
conv_9 = Activation("relu")(conv_9)
conv_10 = Convolution2D(512, (kernel, kernel), padding="same")(conv_9)
conv_10 = BatchNormalization()(conv_10)
conv_10 = Activation("relu")(conv_10)

pool_4, mask_4 = MaxPoolingWithArgmax2D(pool_size)(conv_10)

conv_11 = Convolution2D(512, (kernel, kernel), padding="same")(pool_4)
conv_11 = BatchNormalization()(conv_11)
conv_11 = Activation("relu")(conv_11)
conv_12 = Convolution2D(512, (kernel, kernel), padding="same")(conv_11)
conv_12 = BatchNormalization()(conv_12)
conv_12 = Activation("relu")(conv_12)
conv_13 = Convolution2D(512, (kernel, kernel), padding="same")(conv_12)
conv_13 = BatchNormalization()(conv_13)
conv_13 = Activation("relu")(conv_13)

pool_5, mask_5 = MaxPoolingWithArgmax2D(pool_size)(conv_13)
print("Build enceder done..")

# between encoder and decoder
conv_14 = Convolution2D(512, (kernel, kernel), padding="same")(pool_5)
conv_14 = BatchNormalization()(conv_14)
conv_14 = Activation("relu")(conv_14)
conv_15 = Convolution2D(512, (kernel, kernel), padding="same")(conv_14)
conv_15 = BatchNormalization()(conv_15)
conv_15 = Activation("relu")(conv_15)
conv_16 = Convolution2D(512, (kernel, kernel), padding="same")(conv_15)
conv_16 = BatchNormalization()(conv_16)
conv_16 = Activation("relu")(conv_16)

# decoder
unpool_1 = MaxUnpooling2D(pool_size)(conv_16,mask_5)
concat_1 = Concatenate()([unpool_1, conv_13])

conv_17 = Convolution2D(512, (kernel, kernel), padding="same")(concat_1)
conv_17 = BatchNormalization()(conv_17)
conv_17 = Activation("relu")(conv_17)
conv_18 = Convolution2D(512, (kernel, kernel), padding="same")(conv_17)
conv_18 = BatchNormalization()(conv_18)
conv_18 = Activation("relu")(conv_18)
conv_19 = Convolution2D(512, (kernel, kernel), padding="same")(conv_18)
conv_19 = BatchNormalization()(conv_19)
conv_19 = Activation("relu")(conv_19)

unpool_2 = MaxUnpooling2D(pool_size)(conv_19, mask_4)
concat_2 = Concatenate()([unpool_2, conv_10])

conv_20 = Convolution2D(512, (kernel, kernel), padding="same")(concat_2)
conv_20 = BatchNormalization()(conv_20)
conv_20 = Activation("relu")(conv_20)
conv_21 = Convolution2D(512, (kernel, kernel), padding="same")(conv_20)
conv_21 = BatchNormalization()(conv_21)
conv_21 = Activation("relu")(conv_21)
conv_22 = Convolution2D(256, (kernel, kernel), padding="same")(conv_21)
conv_22 = BatchNormalization()(conv_22)
conv_22 = Activation("relu")(conv_22)

unpool_3 = MaxUnpooling2D(pool_size)(conv_22, mask_3)
concat_3 = Concatenate()([unpool_3, conv_7])

conv_23 = Convolution2D(256, (kernel, kernel), padding="same")(concat_3)
conv_23 = BatchNormalization()(conv_23)
conv_23 = Activation("relu")(conv_23)
conv_24 = Convolution2D(256, (kernel, kernel), padding="same")(conv_23)
conv_24 = BatchNormalization()(conv_24)
conv_24 = Activation("relu")(conv_24)
conv_25 = Convolution2D(128, (kernel, kernel), padding="same")(conv_24)
conv_25 = BatchNormalization()(conv_25)
conv_25 = Activation("relu")(conv_25)

unpool_4 = MaxUnpooling2D(pool_size)(conv_25, mask_2)
concat_4 = Concatenate()([unpool_4, conv_4])

conv_26 = Convolution2D(128, (kernel, kernel), padding="same")(concat_4)
conv_26 = BatchNormalization()(conv_26)
conv_26 = Activation("relu")(conv_26)
conv_27 = Convolution2D(64, (kernel, kernel), padding="same")(conv_26)
conv_27 = BatchNormalization()(conv_27)
conv_27 = Activation("relu")(conv_27)

unpool_5 = MaxUnpooling2D(pool_size)(conv_27, mask_1)
concat_5 = Concatenate()([unpool_5, conv_2])

conv_28 = Convolution2D(64, (kernel, kernel), padding="same")(concat_5)
conv_28 = BatchNormalization()(conv_28)
conv_28 = Activation("relu")(conv_28)

conv_29 = Convolution2D(n_labels, (1, 1), padding="valid")(conv_28)
conv_29 = BatchNormalization()(conv_29)
conv_29 = Activation(output_mode)(conv_29)
outputs = Reshape((256,256,3))(conv_29)

print("Build decoder done..")

segunet = Model(inputs=inputs, outputs=outputs)


Build enceder done..


ValueError: ignored

In [ ]:

def CreateSegUNet(input_shape=(256,256,3), n_labels=3, kernel=3, pool_size=(2, 2), output_mode="softmax"):
    inputs = Input(shape=input_shape)

    # encoder
    conv_1 = Convolution2D(64, (kernel, kernel), padding="same")(inputs)
    conv_1 = BatchNormalization()(conv_1)
    conv_1 = Activation("relu")(conv_1)
    conv_2 = Convolution2D(64, (kernel, kernel), padding="same")(conv_1)
    conv_2 = BatchNormalization()(conv_2)
    conv_2 = Activation("relu")(conv_2)

    pool_1, mask_1 = MaxPoolingWithArgmax2D(pool_size)(conv_2)

    conv_3 = Convolution2D(128, (kernel, kernel), padding="same")(pool_1)
    conv_3 = BatchNormalization()(conv_3)
    conv_3 = Activation("relu")(conv_3)
    conv_4 = Convolution2D(128, (kernel, kernel), padding="same")(conv_3)
    conv_4 = BatchNormalization()(conv_4)
    conv_4 = Activation("relu")(conv_4)

    pool_2, mask_2 = MaxPoolingWithArgmax2D(pool_size)(conv_4)

    conv_5 = Convolution2D(256, (kernel, kernel), padding="same")(pool_2)
    conv_5 = BatchNormalization()(conv_5)
    conv_5 = Activation("relu")(conv_5)
    conv_6 = Convolution2D(256, (kernel, kernel), padding="same")(conv_5)
    conv_6 = BatchNormalization()(conv_6)
    conv_6 = Activation("relu")(conv_6)
    conv_7 = Convolution2D(256, (kernel, kernel), padding="same")(conv_6)
    conv_7 = BatchNormalization()(conv_7)
    conv_7 = Activation("relu")(conv_7)

    pool_3, mask_3 = MaxPoolingWithArgmax2D(pool_size)(conv_7)

    conv_8 = Convolution2D(512, (kernel, kernel), padding="same")(pool_3)
    conv_8 = BatchNormalization()(conv_8)
    conv_8 = Activation("relu")(conv_8)
    conv_9 = Convolution2D(512, (kernel, kernel), padding="same")(conv_8)
    conv_9 = BatchNormalization()(conv_9)
    conv_9 = Activation("relu")(conv_9)
    conv_10 = Convolution2D(512, (kernel, kernel), padding="same")(conv_9)
    conv_10 = BatchNormalization()(conv_10)
    conv_10 = Activation("relu")(conv_10)

    pool_4, mask_4 = MaxPoolingWithArgmax2D(pool_size)(conv_10)

    conv_11 = Convolution2D(512, (kernel, kernel), padding="same")(pool_4)
    conv_11 = BatchNormalization()(conv_11)
    conv_11 = Activation("relu")(conv_11)
    conv_12 = Convolution2D(512, (kernel, kernel), padding="same")(conv_11)
    conv_12 = BatchNormalization()(conv_12)
    conv_12 = Activation("relu")(conv_12)
    conv_13 = Convolution2D(512, (kernel, kernel), padding="same")(conv_12)
    conv_13 = BatchNormalization()(conv_13)
    conv_13 = Activation("relu")(conv_13)

    pool_5, mask_5 = MaxPoolingWithArgmax2D(pool_size)(conv_13)
    print("Build enceder done..")

    # between encoder and decoder
    conv_14 = Convolution2D(512, (kernel, kernel), padding="same")(pool_5)
    conv_14 = BatchNormalization()(conv_14)
    conv_14 = Activation("relu")(conv_14)
    conv_15 = Convolution2D(512, (kernel, kernel), padding="same")(conv_14)
    conv_15 = BatchNormalization()(conv_15)
    conv_15 = Activation("relu")(conv_15)
    conv_16 = Convolution2D(512, (kernel, kernel), padding="same")(conv_15)
    conv_16 = BatchNormalization()(conv_16)
    conv_16 = Activation("relu")(conv_16)

    # decoder
    unpool_1 = MaxUnpooling2D(pool_size)(conv_16, mask_5)
    concat_1 = Concatenate()([unpool_1, conv_13])

    conv_17 = Convolution2D(512, (kernel, kernel), padding="same")(concat_1)
    conv_17 = BatchNormalization()(conv_17)
    conv_17 = Activation("relu")(conv_17)
    conv_18 = Convolution2D(512, (kernel, kernel), padding="same")(conv_17)
    conv_18 = BatchNormalization()(conv_18)
    conv_18 = Activation("relu")(conv_18)
    conv_19 = Convolution2D(512, (kernel, kernel), padding="same")(conv_18)
    conv_19 = BatchNormalization()(conv_19)
    conv_19 = Activation("relu")(conv_19)

    unpool_2 = MaxUnpooling2D(pool_size)(conv_19, mask_4)
    concat_2 = Concatenate()([unpool_2, conv_10])

    conv_20 = Convolution2D(512, (kernel, kernel), padding="same")(concat_2)
    conv_20 = BatchNormalization()(conv_20)
    conv_20 = Activation("relu")(conv_20)
    conv_21 = Convolution2D(512, (kernel, kernel), padding="same")(conv_20)
    conv_21 = BatchNormalization()(conv_21)
    conv_21 = Activation("relu")(conv_21)
    conv_22 = Convolution2D(256, (kernel, kernel), padding="same")(conv_21)
    conv_22 = BatchNormalization()(conv_22)
    conv_22 = Activation("relu")(conv_22)

    unpool_3 = MaxUnpooling2D(pool_size)(conv_22, mask_3)
    concat_3 = Concatenate()([unpool_3, conv_7])

    conv_23 = Convolution2D(256, (kernel, kernel), padding="same")(concat_3)
    conv_23 = BatchNormalization()(conv_23)
    conv_23 = Activation("relu")(conv_23)
    conv_24 = Convolution2D(256, (kernel, kernel), padding="same")(conv_23)
    conv_24 = BatchNormalization()(conv_24)
    conv_24 = Activation("relu")(conv_24)
    conv_25 = Convolution2D(128, (kernel, kernel), padding="same")(conv_24)
    conv_25 = BatchNormalization()(conv_25)
    conv_25 = Activation("relu")(conv_25)

    unpool_4 = MaxUnpooling2D(pool_size)(conv_25, mask_2)
    concat_4 = Concatenate()([unpool_4, conv_4])

    conv_26 = Convolution2D(128, (kernel, kernel), padding="same")(concat_4)
    conv_26 = BatchNormalization()(conv_26)
    conv_26 = Activation("relu")(conv_26)
    conv_27 = Convolution2D(64, (kernel, kernel), padding="same")(conv_26)
    conv_27 = BatchNormalization()(conv_27)
    conv_27 = Activation("relu")(conv_27)

    unpool_5 = MaxUnpooling2D(pool_size)(conv_27, mask_1)
    concat_5 = Concatenate()([unpool_5, conv_2])

    conv_28 = Convolution2D(64, (kernel, kernel), padding="same")(concat_5)
    conv_28 = BatchNormalization()(conv_28)
    conv_28 = Activation("relu")(conv_28)

    conv_29 = Convolution2D(n_labels, (1, 1), padding="valid")(conv_28)
    conv_29 = BatchNormalization()(conv_29)
    conv_29 = Activation(output_mode)(conv_29)
    outputs = Reshape((256,256,3))(conv_29)

    print("Build decoder done..")

    segunet = Model(inputs=inputs, outputs=outputs)

    return segunet

In [ ]:
inputshape = (256,256,3)
Res_Unet_model=CreateSegUNet(inputshape)

Build enceder done..


AttributeError: ignored

In [ ]:
Res_Unet_model.summary()

In [ ]:
class DataGenerator(tf.keras.utils.Sequence):
  def __init__(self, ids , mask, image_dir = './', batch_size = 16, img_h = 256, img_w = 256, shuffle = True):
    self.ids = ids
    self.mask = mask
    self.image_dir = image_dir
    self.batch_size = batch_size
    self.img_h = img_h
    self.img_w = img_w
    self.shuffle = shuffle
    self.on_epoch_end()
  def __len__(self):
    return int(np.floor(len(self.ids)) / self.batch_size)

  def __getitem__(self, index):
    indexes = self.indexes[index* self.batch_size : (index+1) * self.batch_size]
    list_ids = [self.ids[i] for i in indexes]
    list_mask = [self.mask[i] for i in indexes]
    X, y = self.__data_generation(list_ids, list_mask)
    return X, y

  def on_epoch_end(self):    
    self.indexes = np.arange(len(self.ids))

    if self.shuffle:
      np.random.shuffle(self.indexes)

  def __data_generation(self, list_ids, list_mask):

    X = np.empty((self.batch_size, self.img_h, self.img_w, 3))
    y = np.empty((self.batch_size, self.img_h, self.img_w, 3))

    for i in range(len(list_ids)):
      img_path = str(list_ids[i])
      mask_path = str(list_mask[i])
      img = io.imread(img_path)
      mask = io.imread(mask_path)
      img = cv2.resize(img,(self.img_h,self.img_w))
      img = np.array(img, dtype = np.float64)
      mask = cv2.resize(mask,(self.img_h,self.img_w))
      mask = np.array(mask, dtype = np.float64)
      img -= img.mean()
      img /= img.std()
      mask -= mask.mean()
      mask /= mask.std()
      X[i,] = img
      y[i,] = mask
    y = (y > 0).astype(int)
    return X, y

train_data = DataGenerator(train_ids, train_mask)
val_data = DataGenerator(val_ids, val_mask)

In [ ]:
earlystopping = EarlyStopping(monitor='val_loss',
                              mode='min', 
                              verbose=1, 
                              patience=20
                             )
checkpointer = ModelCheckpoint(filepath="/content/drive/MyDrive/SegUnet/SegUnet.h5", 
                               verbose=1, 
                               save_best_only=True
                              )
reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                              mode='min',
                              verbose=1,
                              patience=10,
                              min_delta=0.0001,
                              factor=0.2
                             )
filename='/content/drive/MyDrive/SegUnet/history.csv'
history_logger=tf.keras.callbacks.CSVLogger(filename, separator=",", append=True)

In [ ]:
def dice_coef(y_true, y_pred):
    smooth=100
    y_true = tf.cast(y_true, tf.float32)
    y_truef=K.flatten(y_true)
    y_predf=K.flatten(y_pred)
    And=K.sum(y_truef* y_predf)
    return ((2* And + smooth) / (K.sum(y_truef) + K.sum(y_predf) + smooth))

In [ ]:
def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)

In [ ]:
def iou(y_true, y_pred, smooth = 100):
    intersection = K.sum(y_true * y_pred)
    sum_ = K.sum(y_true + y_pred)
    jac = (intersection + smooth) / (sum_ - intersection + smooth)
    return jac

In [ ]:
adam = tf.keras.optimizers.Adam(lr = 0.01, epsilon = 0.1)
Res_Unet_model.compile(optimizer = adam, 
                  loss = dice_coef_loss, 
                  metrics = [dice_coef,iou,Recall(),Precision()]
                 )



/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


NameError: ignored

In [ ]:
# Res_Unet_model.load_weights("/content/drive/MyDrive/CT_res_att_unet/CT_res_att_unet.h5")

In [ ]:
Res_Unet_model.fit(train_data, 
                  epochs = 100,
                  validation_data = val_data,
                  callbacks = [checkpointer, earlystopping, reduce_lr,history_logger]
                 )

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:44: RuntimeWarning: invalid value encountered in true_divide


Epoch 1/100
